In [0]:
import dlt
import pyspark.sql.functions as F 

In [0]:
@dlt.table(
    name="bronze_orders",
    table_properties={"quality": "bronze"},
    comment="Raw order data from source cloud file",
)
def create_bronze_addresses():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")       
        .load("/Volumes/circuitbox/landing/operational_data/orders")
    ).select(
        "*",
        F.col("_metadata.file_path").alias("input_file_path"),
        F.current_timestamp().alias("ingestion_timestamp"),
    )

In [0]:
@dlt.table(
    name="silver_orders_clean",
    table_properties={"quality": "silver"},
    comment="Cleaned order data",
)
def create_silver_orders():
    return (spark.readStream.table("LIVE.bronze_orders")).select(
        "customer_id",
        "items",
        "order_id",
        "order_status",
        F.col("order_timestamp").cast("timestamp").alias("order_timestamp")
        ,"payment_method"
    )


In [0]:
@dlt.table(
    name="silver_orders",
    table_properties={"quality": "silver"},
    comment="exploded order data",
)
def create_silver_orders():
    return (
        spark.readStream.table("LIVE.silver_orders_clean")
        .withColumn("items", F.explode("items"))
        .select(
            "customer_id",
            "items.category",
            "items.item_id",
            "items.name",
            "items.price",
            "items.quantity",
            "order_id",
            "order_status",
            "order_timestamp",
            "payment_method",
        )
    )